# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119714e+02     1.819000e+00
 * time: 0.08711791038513184
     1     1.132018e+01     9.399134e-01
 * time: 0.8867719173431396
     2    -1.227120e+01     1.116811e+00
 * time: 1.020946979522705
     3    -3.414726e+01     8.094033e-01
 * time: 1.1826889514923096
     4    -4.792017e+01     5.329285e-01
 * time: 1.3282568454742432
     5    -5.707789e+01     1.900926e-01
 * time: 1.4703028202056885
     6    -5.989767e+01     1.278747e-01
 * time: 1.5749108791351318
     7    -6.094767e+01     5.471213e-02
 * time: 1.6859238147735596
     8    -6.134419e+01     6.542203e-02
 * time: 1.79697585105896
     9    -6.160811e+01     4.543255e-02
 * time: 1.9012019634246826
    10    -6.180410e+01     3.337327e-02
 * time: 2.011802911758423
    11    -6.196774e+01     2.380573e-02
 * time: 2.1218748092651367
    12    -6.203892e+01     1.798207e-02
 * time: 2.2249820232391357
    13    -6.210782e+01     1.764160e-02
 * time: 2.3357458114624

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671084
    AtomicLocal         -18.8557743
    AtomicNonlocal      14.8522679
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485396 
    Xc                  -19.3336830

    total               -62.261666455531
